In [19]:
import warnings

import pandas as pd
import numpy as np
import torch

from MLWrappers import KerasBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [20]:
MODEL_NAME = 'nn_keras'
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# We load the data used to train and test the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
train_label = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_label.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)


In [21]:
# we load the target black box model using our wrapper
target = KerasBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.keras')

In [22]:
# We can define the parameters to be passed to the shadow models (in our case random forests)
shadow_forest_params = {'n_estimators': 100}

In [23]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5,
                       shadow_test_size=0.8,
                       undersample_attack_dataset=True)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=5,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=5,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.3,
                         undersample_attack_dataset=True)
attacks = [
    mia,
    label_only,
    aloa
]

In [24]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the list of attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [25]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set, save_folder = f'./results_{DS_NAME}_{MODEL_NAME}')

In [26]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)

{'mia_attack': {'classification_report': {'IN': {'precision': 0.7995314101625421, 'recall': 0.5655979696483141, 'f1-score': 0.662520855452753, 'support': 19307}, 'OUT': {'precision': 0.1994081710576556, 'recall': 0.4327739796975347, 'f1-score': 0.27301836241259886, 'support': 4827}, 'accuracy': 0.5390320709372669, 'macro avg': {'precision': 0.4994697906100989, 'recall': 0.4991859746729244, 'f1-score': 0.46776960893267594, 'support': 24134}, 'weighted avg': {'precision': 0.6795017890819385, 'recall': 0.5390320709372669, 'f1-score': 0.5846171290126757, 'support': 24134}}}}


In [27]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.7995314101625421,
  'recall': 0.5655979696483141,
  'f1-score': 0.662520855452753,
  'support': 19307},
 'OUT': {'precision': 0.1994081710576556,
  'recall': 0.4327739796975347,
  'f1-score': 0.27301836241259886,
  'support': 4827},
 'accuracy': 0.5390320709372669,
 'macro avg': {'precision': 0.4994697906100989,
  'recall': 0.4991859746729244,
  'f1-score': 0.46776960893267594,
  'support': 24134},
 'weighted avg': {'precision': 0.6795017890819385,
  'recall': 0.5390320709372669,
  'f1-score': 0.5846171290126757,
  'support': 24134}}

In [28]:
results['label_only_attack']['classification_report']

KeyError: 'label_only_attack'

In [ ]:
results['aloa_attack']['classification_report']